<a href="https://colab.research.google.com/github/milanamagurina/Transformers-Generative-Model-Training/blob/main/Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install transformers tokenizers datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:

%pip install --upgrade git+https://github.com/dask/s3fs

  Cloning https://github.com/dask/s3fs to /tmp/pip-req-build-7qcnoedp
  Running command git clone --filter=blob:none --quiet https://github.com/dask/s3fs /tmp/pip-req-build-7qcnoedp
  Resolved https://github.com/dask/s3fs to commit 51e3c80ef380a82081a171de652e2b699753be2b
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 102.1 MB/s eta 0:00:00
  Created wheel for s3fs: filename=s3fs-2024.12.0-py3-none-any.whl size=30197 sha256=de35cf82e879516563f2e35458318127ab6b1fb5997f9cc2adc87e69565c1c52
  Stored in directory: /tmp/pip-ephem-wheel-cache-m9jw4py9/wheels/2c/cf/f6/37bf77f428e42d3845ec3f2c308e4a7b05e326dda062fbf438
Successfully built s3fs
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.9.0
    Uninstalling fsspec-2024.9.0:
      Successfully uninstalled fss

In [ ]:
urls = [
    "https://www.gutenberg.org/files/31547/31547-0.txt",
    "https://www.gutenberg.org/files/49819/49819-0.txt",
    "https://www.gutenberg.org/files/49820/49820-0.txt",
    "https://www.gutenberg.org/files/49821/49821-0.txt",
    "https://www.gutenberg.org/files/55738/55738-0.txt",
    "https://www.gutenberg.org/files/68377/68377-0.txt",
]

In [ ]:
import html
import re

import requests


def download(url):
    return requests.get(url).text


striptags_re = re.compile(r"(<!--.*?-->|<[^>]*>)")
entity_re = re.compile(r"&([^;]+);")


def to_text(s):
    return html.unescape(striptags_re.sub("", s))


def beautify(s):
    lines = [x.strip() for x in s.split("\n") if x.strip() != ""]
    for i in range(min(100, len(lines))):
        if lines[i] == "-->":
            break
    return "\n".join(lines[i + 1 :] if i < 100 else lines)


with open("dataset.txt", "w", encoding="utf-8") as f:
    for u in urls:
        text = beautify(to_text(download(u)))
        f.write(text + "\n\n")

In [ ]:
with open("dataset.txt", "r", encoding="utf-8") as file:
    lines = file.readlines()

print(lines[20:40])  # Print the first 20 lines

['For instance, he found that a quantity of hydrogen gas invariably\n', 'combined with eight times its own mass of oxygen gas to form water. He\n', 'guessed that water consisted of combinations of 1 atom of hydrogen with\n', '1 atom of oxygen. (A combination of atoms is called a “molecule” from a\n', 'Greek word meaning “a small mass”, and so hydrogen and oxygen atoms can\n', 'be said to combine to form a “water molecule”.)\n', '[Illustration: _John Dalton_]\n', 'To account for the difference in the masses of the combining gases,\n', 'Dalton decided that the oxygen atom was eight times as massive as the\n', 'hydrogen atom. If he set the mass of the hydrogen atom at 1 (just for\n', 'convenience) then the mass of the oxygen atom ought to be set at 8.\n', 'These comparative, or relative, numbers were said to be “atomic\n', 'weights”, so that what Dalton was suggesting was that the atomic weight\n', 'of hydrogen was 1 and the atomic weight of oxygen was 8. By noting the\n', 'quantity of ot

In [ ]:
import tokenizers as tok
import transformers as tr

In [ ]:
tokenizer = tok.Tokenizer(tok.models.BPE(unk_token = "[UNK]"))
tokenizer.pre_tokenizer = tok.pre_tokenizers.Whitespace()
trainer = tok.trainers.BpeTrainer(special_tokens = ["PAD"])
tokenizer.train(["dataset.txt"], trainer)
tokenizer.enable_padding()

In [ ]:
tokenizer.encode("The quick brown fox jumps over the lazy dog").tokens

['The',
 'quick',
 'brown',
 'fo',
 'x',
 'j',
 'um',
 'ps',
 'over',
 'the',
 'la',
 'zy',
 'do',
 'g']

As we can see - popular words has been tokenized in whole word, however ones that uses rare in text or does not apprear - divided into fragments.


In [ ]:
vocab = tokenizer.get_vocab()
ttokenizer = tr.PreTrainedTokenizerFast(tokenizer_object = tokenizer)
len(vocab)

10507

In [ ]:
config = tr.GPT2Config(
    vocab_size=len(vocab),
    bos_token_id=tokenizer.token_to_id("[CLS]"),
    eos_token_id=tokenizer.token_to_id("[EOS]"),
)
gpt = tr.GPT2LMHeadModel(config)


The weights of the newly created model are initialized randomly, so if we ask such a model to generate text, it will produce nonsense.

In [ ]:
res = gpt.generate(
    **ttokenizer("The space  ", return_tensors = "pt"),
    max_length = 100,
    top_k = 3,
    do_sample = True
)

ttokenizer.decode(res[0])


'The space ER investig thal thal thal ³⁷ ³⁷ ³⁷ ER ER ER ER ER ER ³⁷ ³⁷ 1910 1910 ER ER ER Weapons_ POR bearing ER ER 1910 1910 Svante Svante conscious conscious conscious osmic y_ y_ y_ industr industr ER ER ER ER batter batter drogen recently recently captions captions captions POR preventing ³⁷ influence drogen penned POR xed POR ³⁷ ³⁷ Marie dangers ER ER ER ER Fair Chal Chal penned penned penned penned penned Spontaneous ER ER drive Times_ Times_ ER 1910 hysic ER ER 1910 Heisenberg_ Harrisburg ER ER 41 Fair actly actly CLE ER'

In [ ]:
import datasets

dataset = datasets.load_dataset("text", data_files="dataset.txt")
dataset["train"][13]

Generating train split: 0 examples [00:00, ? examples/s]

{'text': 'element. The chief difference between the various atoms lay in their'}

In [ ]:
def tokenize(x):
    x = ttokenizer(x["text"])
    x["labels"] = x["input_ids"].copy()
    return x

ds = dataset.map(tokenize, batched = True, remove_columns=["text"])
ds["train"][0]

Map:   0%|          | 0/5458 [00:00<?, ? examples/s]

{'input_ids': [], 'token_type_ids': [], 'attention_mask': [], 'labels': []}

In [ ]:
from itertools import chain

block_size = 1024

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

dsb = ds.map(group_texts, batched=True)

Map:   0%|          | 0/5458 [00:00<?, ? examples/s]

In [ ]:
targs = tr.TrainingArguments(
    output_dir = "gpt2_scratch",
    num_train_epochs = 3,
    learning_rate = 5e-5,
    warmup_steps = 200,
    save_steps = 5000,
    per_device_train_batch_size=2,
)

trainer = tr.Trainer(
    gpt,
    args = targs,
    train_dataset = dsb["train"],
    tokenizer = ttokenizer,
    data_collator = tr.default_data_collator,
)


<ipython-input-16-75896748caf8>:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = tr.Trainer(


In [ ]:
'''import os
os.environ["WANDB_DISABLED"] = "true"  # Disable W&B tracking'''


'import os\nos.environ["WANDB_DISABLED"] = "true"  # Disable W&B tracking'

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss


TrainOutput(global_step=93, training_loss=7.970216812626008, metrics={'train_runtime': 117.7411, 'train_samples_per_second': 1.554, 'train_steps_per_second': 0.79, 'total_flos': 95632883712000.0, 'train_loss': 7.970216812626008, 'epoch': 3.0})

In [ ]:
trainer.save_model("gpt2_scratch")  # Saves the model, config, and tokenizer
ttokenizer.save_pretrained("gpt2_scratch")  # Save tokenizer files

print("Model and tokenizer saved successfully to 'gpt2_scratch'")

Model and tokenizer saved successfully to 'gpt2_scratch'


In [ ]:
def save_training_args(targs, filepath):
    import json
    with open(filepath, "w") as f:
        json.dump(targs.to_dict(), f, indent=4)


In [ ]:
save_training_args(targs, "gpt2_scratch/training_args.json")


In [ ]:
#Reload saved weigths and parameters
'''
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model = GPT2LMHeadModel.from_pretrained("gpt2_scratch")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2_scratch")'''


In [ ]:
res = gpt.generate(
    **ttokenizer("Humanity's greatest invention ", return_tensors="pt").to("cuda"),
    max_new_tokens=150,
    do_sample=True
)
ttokenizer.decode(res[0])

"Human ity ' s greatest inv ention of the so The and cell of the , a are that be of - and , are . be . The , to in the most . nuclear are of the atoms of the . The , more - . The most , a . The of and to only are of the , on , that the , . The from was of the uranium be , , for , through . In , on . The . , by , s a of the , of the . The of the in be , . The , of the and , a to the most , energy of into in the and was of the , , a , and , and was the that , the . The . in be , are that , a 2 that of the , and that a , ,"

In [ ]:
from transformers import AutoTokenizer, GPT2LMHeadModel

model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
input_ids = tokenizer("Humanity ", return_tensors="pt").input_ids

output = model.generate(
    input_ids=input_ids,
    max_new_tokens=150,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.7
)
decoded_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(decoded_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Humanity  (p. 6)
The Bible teaches that God is the Word of God, and the Word of God is God Himself. God is the Word of God, the Word of God is the Word of God, and God is the Word of God. God is the Word of God, the Word of God is the Word of God, and God is the Word of God. God is the Word of God, the Word of God is the Word of God, and God is the Word of God. God is the Word of God, the Word of God is the Word of God, and God is the Word of God. God is the Word of God, the Word of God is the Word of God, and God is the Word of


In [32]:
# Encode input with attention mask
inputs = tokenizer(
    "Humanity ",
    return_tensors="pt",
    truncation=True,
    max_length=512,
)
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# Generate text with attention mask and pad_token_id
output = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_new_tokens=150,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id
)
decoded_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(decoded_text)


Humanity  is not a religion. The main problem with the idea of a religious state in a modern world is that it is completely absent from the reality of the world. It is not a religion. The main problem with the idea of a religious state in a modern world is that it is completely absent from the reality of the world. It is not a religion.
This is why the term "religious state" is such a good name, because it is not so much a religion as a political system. The idea of a religious state is not something that can be understood in terms of a political system. It is a political system that can be understood as a political system that is based on religious belief. The idea of a religious state is not something
